In [1]:

# Colab-specific setup
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
Organize your dataset like:

kotlin
Copy
Edit
data/
  train/
    fresh/
    rotten/
  val/
    fresh/
    rotten/
📸 2. Image Data Pipeline
python
Copy
Edit
IMG_SIZE = (224, 224)
BATCH = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    validation_split=0.2
)

train_gen = train_datagen.flow_from_directory(
    '/content/data/train',
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='binary',
    subset='training'
)
val_gen = train_datagen.flow_from_directory(
    '/content/data/train',
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='binary',
    subset='validation'
)
🤖 3. Transfer Learning Model (e.g., VGG16 / MobileNetV2)
python
Copy
Edit
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

base = VGG16(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
base.trainable = False  # freeze base

model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
🏃 4. Train & Fine-Tune
python
Copy
Edit
epochs = 10
history = model.fit(train_gen, validation_data=val_gen, epochs=epochs)

# Optionally: fine-tune some layers
base.trainable = True
for layer in base.layers[:-4]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_gen, validation_data=val_gen, epochs=5)
📊 5. Evaluate and Predict
python
Copy
Edit
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(acc, label='train'); plt.plot(val_acc, label='val'); plt.legend()
For predictions:

python
Copy
Edit
def predict_image(img_path):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(img_path, target_size=IMG_SIZE)
    x = image.img_to_array(img)/255.0
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)[0][0]
    print("Fresh" if pred >= 0.5 else "Rotten")







SyntaxError: invalid character '📸' (U+1F4F8) (ipython-input-1-1949469836.py, line 20)